# Imports etc

In [15]:
# imports
%cd /content/drive/My\ Drive/Thesis/
from NNClasses import *
from NNDataLoader import *
from NN_train_util import *
from util import *
import os
import pickle
from copy import deepcopy
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
from matplotlib import cm
from tqdm.auto import tqdm
from time import time, sleep
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.optim as optim
from sklearn.model_selection import KFold

%load_ext autoreload
%autoreload 1
%aimport NNClasses
%aimport NNDataLoader
%aimport NN_train_util

# ----------------------------
# FIXED TRAINING PARAMETERS
# ----------------------------

early_stop_patience = 50
Tc0 = .894 #from "Critical behavior of the two-dimensional XYmodel", Gupta & Baillie, 1992
early_stopping_enabled = True # True  
criterion = nn.BCELoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"torch device: {device}")

/content/drive/My Drive/Thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
torch device: cuda


# Load julia configs and save CustomDataset

In [18]:
L_range = [8,16,24,32,40,48,56,64] 
T_range = np.array([0.1,0.14603174603174604,0.19206349206349208,0.23809523809523808,0.2841269841269841,0.33015873015873015,0.3761904761904762,0.4222222222222222,0.46825396825396826,0.5142857142857142,0.5603174603174603,0.6063492063492063,0.6523809523809524,0.6984126984126984,0.7444444444444445,0.7904761904761904,0.8365079365079365,0.8825396825396825,0.9285714285714286,0.9746031746031746,1.0206349206349206,1.0666666666666667,1.1126984126984127,1.1587301587301588,1.2047619047619047,1.2507936507936508,1.2968253968253969,1.3428571428571427,1.3888888888888888,1.434920634920635,1.480952380952381,1.5269841269841269,1.573015873015873,1.619047619047619,1.6650793650793652,1.711111111111111,1.7571428571428571,1.8031746031746032,1.8492063492063493,1.8952380952380952,1.9412698412698413,1.9873015873015873,2.033333333333333,2.0793650793650795,2.1253968253968254,2.1714285714285713,2.2174603174603176,2.2634920634920634,2.3095238095238093,2.3555555555555556,2.4015873015873015,2.447619047619048,2.4936507936507937,2.5396825396825395,2.585714285714286,2.6317460317460317,2.6777777777777776,2.723809523809524,2.7698412698412698,2.815873015873016,2.861904761904762,2.907936507936508,2.953968253968254,3.0])
Ts = np.repeat(T_range, 100) # array([0.1, 0.1, 0.1, ..., 3. , 3. , 3. ])

root_test = os.path.join("data", "test")
for L in L_range:
  save_path = os.path.join(root_test, f"dataset_julia_L={L}")
  if not os.path.isfile(save_path): # only do this once
    # load cfgs
    root_cfgs = os.path.join(root_test, "test-100", f"L={L}", "configs.npy")
    cfgs = np.load(root_cfgs)
    cfgs = cfgs.reshape((cfgs.shape[0],L,L))

    # test data
    energies = [calcAvgEnergy(cfg) for cfg in cfgs]
    plt.scatter(range(len(energies)),energies, 0.005)

    # julia generated dataset is normalized (spin angles) by default
    test_set = CustomDataset.usingData(L, cfgs, Ts, normalize=True)

    # pickle CustomDataset
    with open(save_path, "wb") as f:
      pickle.dump(data_set, f)